In [1]:
from ctrade import *
from plot import *
from datetime import datetime
%matplotlib inline

In [2]:
p = Poloniex()

In [3]:
major = ['ETH', 'LTC', 'XRP']

In [4]:
pairs = [i for i in p.currency_pairs if any([j for j in major if 'USDT_'+j in i])]
pairs += [i for i in p.currency_pairs if any([j for j in major if 'BTC_'+j in i])]

In [5]:
pairs

['USDT_ETH', 'USDT_LTC', 'USDT_XRP', 'BTC_ETH', 'BTC_LTC', 'BTC_XRP']

In [72]:
from collections import OrderedDict
inf = lambda x: np.isinf(x)

def build_dataset(currency_pairs, days_back, period):
    out = {}
    for i in currency_pairs:
        completed = False
        while not completed:
            try:
                t = p.chart(i, days_back, period).df
                completed = True
            except:
                pass
        print(i)
#         t[i] = (t['close'] - t['close'].shift(1))/t['close']
        last = (t['close'].iloc[-1] - t['open'].iloc[-1])/t['open'].iloc[-1]*100
        last_day =  (t['close'].iloc[-1] - t['open'].iloc[-7])/t['open'].iloc[-7]*100
        out[i] = (last, last_day)
        
    return OrderedDict(sorted(out.items(), key=lambda x: x[1][1], reverse=True))

In [73]:
out = build_dataset(pairs, 1, '4h')

USDT_ETH
USDT_LTC
USDT_XRP
BTC_ETH
BTC_LTC
BTC_XRP


In [75]:


print print_update(out)

Crypto-update 2017-06-12 23:10
BTC/ETH: 	last 4 hours = +5.08 - last day = +21.8
USDT/ETH: 	last 4 hours = +10.49 - last day = +6.36
BTC/XRP: 	last 4 hours = -0.17 - last day = +1.13
BTC/LTC: 	last 4 hours = -0.13 - last day = +0.14
USDT/XRP: 	last 4 hours = +4.77 - last day = -12.1
USDT/LTC: 	last 4 hours = +5.41 - last day = -12.83



In [9]:
plt.scatter(out['USDT_ETH_PC'], out['USDT_BTC_PC'])

KeyError: 'USDT_BTC_PC'

In [ ]:
plt.scatter(out['BTC_BCN_PC'], out['USDT_BTC_PC'])

In [ ]:
plt.scatter(out['BTC_XRP_PC'], out['USDT_BTC_PC'])

In [ ]:
def consecutive_periods(df, columns):
    if not isinstance(columns, list):
        columns = list(columns)
    for col in columns:
        y = df[col].apply(lambda x: 1 if x>0 else 0)
        y = y * (y.groupby((y != y.shift()).cumsum()).cumcount() + 1)
        df[col+'_cdw'] = y
    return df

In [ ]:
out = consecutive_periods(out, [i for i in out.columns])

In [ ]:
def get_trainingset(df, currency):
    
    X = df.copy()
    currency_col = [i for i in X.columns if currency in i]
    rest_col = list(set(X.columns) - set(currency_col))

    Y = X[currency_col]
    X = X[rest_col]
    return X, Y
    

In [ ]:
label = 'XRP'
pair = 'BTC_{}_PC'.format(label)

In [ ]:
X, Y = get_trainingset(out, label)
Y = Y[pair]

Y = pd.Series(Y.iloc[1:].values, index=Y.index[:-1])
mask0 = Y<-0.005
mask1 = Y>0.005

Y.loc[mask0] = 0
Y.loc[mask1] = 1

Y = Y[Y.isin([1, 0])]

X = X[X.index.isin(Y.index)]

print X.shape, Y.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from model_utils import *
from preprocessing import *

if len(Y.unique())>2:
    est = RandomForestRegressor(n_estimators=50, min_samples_leaf=10, max_depth=4)
    est = GradientBoostingRegressor(n_estimators=100, min_samples_leaf=10, max_depth=4)
else:
    est = RandomForestClassifier(n_estimators=50, min_samples_leaf=10, max_depth=4)
    est = GradientBoostingClassifier(n_estimators=100, min_samples_leaf=10, max_depth=4)   

res = do_easy_crossval(est, X, Y, folds=10, refit=True)

In [ ]:
pred = res[0]
est = res[1]

fig, ax = plt.subplots(1,1)
# plt.scatter(pred['true'], pred['pred'])
# np.corrcoef(pred['true'], pred['pred'])[0][1]

mask = pred['true'] == 0
pred.loc[mask, 'proba'].plot(kind='hist', ax=ax, range=(0,1), bins=20, alpha=0.3)
mask = pred['true'] == 1
pred.loc[mask, 'proba'].plot(kind='hist', ax=ax, range=(0,1), bins=20, alpha=0.3)

In [ ]:
plot_feature_importances(est, X.columns, top=20)

In [ ]:
pd.ewma?